## 문제 7: 버스 승하차 인원 월별 정류장별 인원

서울버스 노선별, 정류장별 승하차인원을 수집할 수 있는 ```CardSubwayTime API```가 있다.
서울버스 대상 (서울시내, 서울광역, 서울마을)으로, 교통카드(선후불교통카드)를 이용하여
매일 버스노선별로 각 정류장에 승/하차한 인원을 월별로 집계하고 있다.

```CardSubwayTime API``` 호출에 필요한 인자를 정리하면 다음과 같다.
 
변수명 | 타입 | 설명 
-----|-----|-----
KEY | String | 발급받은 인증키
TYPE | String | xml, xmlf, xls, json 데이터 형식
SERVICE | String | 서비스명
START_INDEX | Integer | 페이징 시작번호
END_INDEX | Integer | 페이징 끝번호
USE_MON | String | 사용월	YYYYMM형식의 문자열
BUS_ROUTE_NO | String(선택) |	버스노선번호

	4479426153736b6539356d70667943

### 정규식으로 XML 파싱

XML 파싱을 통해, 원하는 결과를 추출할 수 있다.
여기서는 정규식으로 해보자.
<BUS_STA_NM>(.+?)</BUS_STA_NM>는 태그 안의 괄호안 패턴을 추출하게 된다.

In [139]:
import os
from src import mylib

keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
key=mylib.getKey(keyPath)
_key=str('4479426153736b6539356d70667943')


In [140]:
import requests
import re
busstopurl='http://openAPI.seoul.go.kr:8088/'+_key+'/xml/CardBusTimeNew/1/5/202107/7016'
data=requests.get(busstopurl).text
print(busstopurl)

http://openAPI.seoul.go.kr:8088/{'dataseoul': '46416e4957736b653132314d445a6347'}/xml/CardBusTimeNew/1/5/202107/7016


In [ ]:
print (data)

### 정규식으로 XML 파싱

XML 파싱을 통해, 원하는 결과를 추출할 수 있다.
여기서는 정규식으로 해보자.
<BUS_STA_NM>(.+?)</BUS_STA_NM>는 태그 안의 괄호안 패턴을 추출하게 된다.

In [11]:
p=re.compile('<BUS_STA_NM>(.+?)</BUS_STA_NM>')
res=p.findall(data)
for item in res:
    print (item)

은평공영차고지
월드컵경기장북측
상암초등학교
월드컵파크2단지.에스플렉스센터
월드컵파크3단지.난지천공원


```<.*_NUM>(\d+)</.*_NUM>```는 _NUM 태그 안의 숫자 데이터를 추출하는 정규식이다.

In [12]:
p=re.compile('<.*_NUM>(\d+)</.*_NUM>')
res=p.findall(data)
#print(" ".join(res))
for item in res:
    print (item, end=", ") # connecting numbers with a comma

0, 0, 0, 0, 0, 0, 0, 0, 23, 1, 11, 0, 28, 0, 119, 1, 85, 0, 53, 1, 23, 0, 29, 0, 28, 1, 18, 0, 26, 0, 26, 2, 31, 1, 76, 1, 43, 3, 27, 2, 12, 2, 10, 0, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 29, 0, 3, 1, 9, 6, 23, 75, 47, 125, 27, 35, 33, 33, 19, 38, 25, 29, 24, 48, 15, 61, 39, 44, 56, 56, 49, 68, 69, 101, 58, 25, 18, 33, 27, 25, 21, 21, 0, 0, 3, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 13, 192, 39, 763, 27, 435, 12, 282, 26, 238, 25, 212, 35, 182, 27, 227, 19, 246, 31, 214, 18, 254, 30, 319, 23, 265, 31, 189, 24, 127, 21, 158, 6, 70, 1, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 6, 576, 9, 2095, 5, 1215, 14, 522, 13, 295, 9, 275, 17, 297, 8, 261, 9, 257, 18, 220, 22, 212, 85, 232, 30, 244, 26, 150, 10, 139, 26, 168, 2, 70, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 2, 32, 9, 52, 7, 46, 5, 41, 4, 56, 8, 47, 10, 70, 10, 71, 11, 79, 8, 47, 7, 64, 7, 79, 3, 82, 3, 58, 2, 52, 2, 66, 0, 27, 

## 문제 8: 서울시 골목상권 분석

어떤 곳에 입점을 할 경우, 배후 상권에 대해 알 필요가 있다.
주변에 얼마나 많은 사람들이 다니는지,
어떤 직장이 있는지,
관련한 점포는 있는지가 궁금할 수 있다.
이러한 골목상권에 대한 정보를 수집할 수 있다.
제공하는 골목상권분석정보는:
- 서울시 우리마을가게 상권분석서비스(상권-생활인구)
- 서울시 우리마을가게 상권분석서비스(상권배후지-집객시설)
- 서울시 우리마을가게 상권분석서비스(상권배후지-생활인구)
- 서울시 우리마을가게 상권분석서비스(상권-집객시설)
- 서울시 우리마을가게 상권분석서비스(상권배후지-상주인구)
- 서울시 우리마을가게 상권분석서비스(상권_상주인구)
- 서울시 우리마을가게 상권분석서비스(상권배후지-아파트)
- 서울시 우리마을가게 상권분석서비스(상권배후지-점포)
- 서울시 우리마을가게 상권분석서비스(상권-상권변화지표)
- 서울시 우리마을가게 상권분석서비스(행정동별 상권변화지표)
- 서울시 우리마을가게 상권분석서비스(상권-추정매출)
- 서울시 우리마을가게 상권분석서비스(상권-직장인구)
- 서울시 우리마을가게 상권분석서비스(상권배후지-소득소비)
- 서울시 우리마을가게 상권분석서비스(상권배후지-직장인구)
- 서울시 우리마을가게 상권분석서비스(자치구별 상권변화지표)
- 서울시 우리마을가게 상권분석서비스(상권-아파트)
- 서울시 우리마을가게 상권분석서비스(상권영역)
- 서울시 우리마을가게 상권분석서비스(상권-점포)

# 아래부터는 공공 데이터 문제

## 문제 9: 공공데이터포털 서울시 구별 교통사고 사망자 수

공공데이터포털에서 사망교통사고 ```AccidentDeath API```를 사용해서 교통사고 사망자 수를 구해보자.
이 데이터는 1년에 1회 갱신되는, 년별 집계를 한다.
사망교통사고 API는 다음 2가지가 있다.
- ```getRestTrafficAccidentDeath``` 사망교통사고정보 Rest 조회
- ```getWMSTrafficAccidentDeath``` 사망교통사고정보 WMS 조회 (PNG 이미지를 조회)

```AccidentDeath API``` 호출에 필요한 인자는 다음과 같다.

변수명 | 설명 
-----|-----
serviceKey | 공공데이터포털에서 발급받은 인증키
searchYear | 조회하려는 연도 (예: 2019)
siDo | 법정동 시도코드
guGun | 법정동 시군구코드
type | 결과형식 (xml, json)
numOfRows | 한 페이지 결과 수 (Default: 10)
pageNo | 페이지 번호 (Default: 1)

## 문제 10: 공공데이터포털 대기 오염 정보

공공데이터포털에서 대기오염을 검색하면 ```한국환경공단_에어코리아_대기오염정보```를 찾을 수 있다.
이 API를 사용하려면, ```한국환경공단_에어코리아_대기오염정보``` 항목을 선택하여 활용신청'한다.

변수명 | 설명 
-----|-----
serviceKey | 공공데이터포털에서 발급받은 인증키
returnType (선택) | 데이터표출방식, xml 또는 json
numOfRows (선택) | 한 페이지 결과 수 (Default: 10)
pageNo (선택) | 페이지 번호 (Default: 1)
searchDate (선택) | 조회날짜 (예: 2020-11-14)
InformCode (선택) | 통보코드검색 (PM10, PM25, O3) (예: PM10)

# 여기부터 spark RDD


#### SparkSession 생성


Spark를 사용하려면 ```SparkSession``` 객체를 생성해야 한다.

SparkSession을 생성해 보자. SparkSesion은 sql 모듈로 **'pyspark.sql.SparkSession'**을 클라이언트로 사용한다.
필요한 설정은 SparkSession이 만들지기 전에 해 두어야 한다. 여기서는 설정을 별도로 하지 않고 비워 놓았다.
SparkSession은 **builder.getOrCreate()** 함수를 호출하여, 기존의 session 또는 새로 생성하여 사용한다. 함수 **getOrCreate()** 함수는 **singleton 패턴**으로 한 번에 하나의 세션만이 존재하도록 한다.
SparkSession을 종료하려면 stop() 함수를 호출한다.

```python
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark.stop()
```

Spark를 실행하기 전 **필수적**으로 설정해야 하는 항목은
* **master**: (1) 분산의 경우 master URL 또는 (2) 로컬인 경우 ```local[]```라고 적어준다.
즉 local의 수는 CPU core의 수를 의미한다. 예를 들어 ```local[*]```는 가능한 최대한의  core를 사용한다는 의미이다.
예를 들어, local[5]라고 하면, core의 수가 2개라고 하더라도 데이터는 5개의 partitions로 나누어져 주어진다.
    * ```local```은 Spark를 로컬에서 실행한다는 의미이다.
    * ```local[n]```는 worker의 쓰레드를 n개로 한다는 의미. CPU core의 개수에 맞추어 설정하자.
    * ```local[*]``` 는 가능하면 가용한 모든 쓰레드를 사용한다는 의미 (Runtime.getRuntime.availableProcessors()로 그 수를 알 수 있다)
* **appName**: 앱의 이름

In [13]:
import pyspark

spark 변수를 만들어야한다

In [14]:
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [15]:
print ("Spark version \t: {}".format(spark.version))
print ("Spark App \t: {}".format(spark.conf.get('spark.app.name')))
print ("Spark Master \t: {}".format(spark.conf.get('spark.master')))
print ("Spark Host \t: {}".format(spark.conf.get('spark.driver.host')))

Spark version 	: 3.1.2
Spark App 	: myApp
Spark Master 	: local
Spark Host 	: 172.30.1.26


# spark가 생성되었다.

## S.3 데이터 구조

Spark에서는 RDD, Dataframe, DataSet 세 가지 데이터구조를 제공하고 있다. 이 가운데 **RDD API에 대한 지원은 축소**되고 있다.
* **RDD**는 Spark 1.0부터 사용되었고, 이를 기반으로 다른 데이터구조가 만들어졌다. RDD는 데이터가 **비구조적**인 경우 사용하기 적합하다. 모델schema를 정하지 않고 사용할 수 있다.
* **Dataframe**은 버전 1.3에서 제공되어 많이 쓰이고 있다. DataFrame과 DataSet은 데이터가 schema와 데이터타잎을 가진 **구조적**인 경우 사용한다.


데이터구조 | 언제 Spark에 도입 | 설명
---------|---------|---------
**RDD** | 1.0 | **비구조적**, schema free, low-level
**Dataframe** | 1.3 | **구조적**, schema를 가진다. Dataset[Row]와 같은 의미로, 타잎을 강제하지 않는다.
**Dataset** | 1.6 | 자바의 Generic과 같이 Dataset[T]으로 '타잎'을 강제하는 형식이다. Scala and Java에서 사용한다. Python loosely-typed이므로 사용하지 않는다.

* Spark의 RDD, DataFrame 모두 immutable이라 일단 생성되고 나면 원본을 수정할 수 없다.
* Spark의 데이터는 모두 **lazy**, 실제 transformation을 action까지 연기한다. **변환할 때마다 실제 변환이 일어나면 그 결과가 메모리에 저장되는 비효율성**을 막기 위해, **action이 실행되는 경우, 계산이 이루어지고, 실제 메모리를 사용**한다. RDD의 경우, action이 실행될 때마다 재계산이 이루어지는 것을 막기 위해 persist (or cache)함수를 사용할 수 있다.


## S.4 RDD 소개

RDD (Resilient Distributed Dataset)는 그 줄임말에서 알 수 있듯이 **데이터를 저장하고 있는 DataSet**이며, 여러 컴퓨터에 **분산**해서 사용해서 사용할 수 있다는 점이 특징이다.
* Resilient - 작업이 실패하지 않도록 fault tolerent, 즉 어느 한 노드에서 작업이 실패하면 다른 노드에서 실행
* Distributed - 클라스터로 구성된 여러 노드에 분산해서 처리
* Dataset - 데이터 구조

RDD는 Python List, 파일, hdfs 등 다양한 자료에서 생성할 수 있고, 생성된 자료는 수정할 수 없는 read-only이다.|

## S.5 RDD 생성

RDD는 **sparkContext**로부터 만들어 진다.
* 1) 이미 만들어진 배열과 같에서 읽어서 생성한다. 이 경우 parallelize() 함수를 사용하게 된다.
* 2) 또는 파일, 데이터베이스 등 외부에서 읽어서 생성할 수도 있다. textFile() 함수를 사용한다.

생성 방법 | 설명 | 함수
----------|----------|----------
내부에서 읽기 | Pytho list에서 생성 | parallelize()
외부에서 읽기 | 파일, HDFS, HBase 등 | ```textFile("mydir/")```<br>```textFile("mydir/*.txt")```<br>```textFile("mydir/*.gz")```<br>```Hadoop InputFormat```


### List에서 RDD 생성하기

```sparkContext.parallelize()``` 함수를 사용하여 Python list에서 RDD를 생성할 수 있다.

In [16]:
myList=[1,2,3,4,5,6,7]

In [17]:
myRdd1 = spark.sparkContext.parallelize(myList)

take(number)는 num만큼의 줄을 출력한다.

In [18]:
myRdd1.take(3)

[1, 2, 3]

### 파일에서 RDD 생성하기

파일에서 직접 RDD를 생성해 본다.
현재 작업 디렉토리 아래에 **'data/' 디렉토리**를 만들고 아래 파일을 생성한다.
파일 내용은 wikipedia에서 Apache spark를 검색한 후 첫 문단을 복사해서 가져 왔다.
일부러 3째줄은 한글, 4째 줄은 같은 단어를 반복해서 추가했다.

In [20]:
%%writefile data/ds_spark_wiki.txt
Wikipedia
Apache Spark is an open source cluster computing framework.
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
Apache Spark Apache Spark Apache Spark Apache Spark
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
Originally developed at the University of California, Berkeley's AMPLab,
the Spark codebase was later donated to the Apache Software Foundation,
which has maintained it since.
Spark provides an interface for programming entire clusters with
implicit data parallelism and fault-tolerance.

Writing data/ds_spark_wiki.txt


파일에서 RDD를 생성하기 위해서는 앞서와 같이 SparkContext를 사용한다.
파일명을 textFile() 함수 인자로 넣어서 만들어 주면 된다.

외부에서 생성할때는 textFile이다. 
\는 명령어가 두번째 줄로 만들어진다는 뜻

In [21]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

```first()```는 첫 데이터만 조회하는 action함수이다.
first()은 take(1)과 동일한 결과를 출력하는데, 그 이유는 first()는 내부적으로 take(1) 함수를 사용하기 때문이다.

In [22]:
myRdd2.first()

'Wikipedia'

#### RDD와 Spark Dataframe를 만드는 함수는 서로 다르다

DataFrame은 다음 장에서 배우게 되겠지만, file에서 읽는 방식이 RDD와 Dataframe이 서로 다르다.
RDD는 sparkContext.textFile(), Dataframe은 read.text()을 사용한다.

구분 | 설명
-----|-----
SparkSession.sparkContext.textFile() | **'SparkContext'를 사용하므로 RDD를 생성**한다.
SparkSession.read.text() | **DataFrame을 생성**한다.

os.path.join ->운영체제 독립적으로 경로명을 합성해주는 함수

In [23]:
import os
myDf=spark.read.text(os.path.join("data", "ds_spark_wiki.txt"))
print (myDf.first())

Row(value='Wikipedia')


읽고 생성된 변수 myDf의 데이터타입을 type()으로 확인하면 DataFrame이다.

In [24]:
print (type(myDf))

<class 'pyspark.sql.dataframe.DataFrame'>


#### csv에서 RDD 생성하기

csv 파일은 컴마로 구분된 데이터를 저장하고 있다. 이 파일을 읽어서 RDD를 생성해 보자.

In [25]:
%%writefile ./data/ds_spark_2cols.csv
35, 2
40, 27
12, 38
15, 31
21, 1
14, 19
46, 1
10, 34
28, 3
48, 1
16, 2
30, 3
32, 2
48, 1
31, 2
22, 1
12, 3
39, 29
19, 37
25, 2

Overwriting ./data/ds_spark_2cols.csv


파일에서 읽어 RDD를 생성할 수 있다.

In [26]:
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

In [30]:
myList=myRdd4.take(5)
print (type(myList))

<class 'list'>


## S.6 RDD API

앞서 RDD를 생성하여 보았다.
이제는 RDD라는 데이터구조에서 데이터를 읽고 변환하고 분석하여 보자.
RDD는 **데이터 변환Transformations**, **연산Actions**으로 구분할 수 있다.
다음에 배우게 될 **Dataframe**의 **Transformer**, **Estimator**와 비교될 수 있다.

### 변환 **transformations**

변환함수는 **lazy연산**을 한다. 실제 변환은 action이 수행되는 시점까지 늦추어져서 이루어진다.
변환 결과는 RDD 또는 seq(RDD)로 만들어진다.

함수 | 설명 | 예제
-------|-------|-------
```map(fn)``` | 요소별로 fn을 적용해서 결과 RDD 돌려줌 | ```.map(lambda x: x.split(' ')```
```filter(fn)``` | 요소별로 선별하여 fn을 적용해서 결과 RDD 돌려줌 | ```.filter(lambda x: "Spark" in x)```
```flatMap(fn)``` | 요소별로 fn을 적용하고, flat해서 결과 RDD 돌려줌 | ```.flatMap(lambda x: x.split(' '))```
```groupByKey()``` | key를 그룹해서 iterator를 돌려줌. |

flatmap -> map 함수와 다르다 
2중구조 가지고 있을때 그 구조 없애서 처리한다. 납작하게 한다.

### **actions**

RDD를 값으로 변환하는데, 보통 Python 리스트가 생성된다.

함수 | 설명 | 예제
-------|-------|-------
```reduce(fn)``` | 요소별로 fn을 사용해서 줄여서 결과 list를 돌려줌 | ```reduce(lambda x,y:x+y)```
```collect()``` | 모든 요소를 결과 list로 돌려줌 |
```count()``` | 요소의 갯수를 결과 list로 돌려줌 |
```take(n)``` | ```collect()```는 전체이지만, n개만 돌려줌 | ```take(1)```
```countByKey()``` | key별 갯수를 세는 함수 | ```countByKey().items()```
```foreach(fn)``` | 각 데이터 항목에 함수fn을 적용 |

###  S.6.1 우선 Python 함수로 해보기

#### map()함수를 사용하지 않고 해보기

map()을 사용하지 않고 섭씨를 화씨로 변환하는 **c2f()**함수를 만들어 보자.
* 데이터를 하나씩 읽어 **for문으로 처리하고, 리스트로 만들어 반환**한다.
* 나중에 **map()함수를 사용하면 for문이 없어도** 가능하다.

map을 쓰면 for문 없어진다

In [34]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c):
    f=list()
    for i in c:
        _f=(float(9)/5)*i + 32
        f.append(_f)
    return f

print (c2f(celsius))

[102.56, 97.7, 99.14, 100.03999999999999]


 #### map() 함수를 사용해 보기

Python은 map(), reduce(), filter() 함수를 이미 가지고 있다.
Python map()을 사용해 보자. **for문이 사라진다**는 점에 유의한다.
반복문이 외관상으로는 보이지 않지만, map() 함수 내부적으로 실행된다는 점에 유의하자.
**map() 함수의 인자는 2개** 이다.
* (1) 첫째 인자는 **처리 함수**이고, **함수의 return은 반드시 있어야** 한다,
* (2) 두 번째 인자는 **처리하려는 데이터**이다.

함수 | 설명 | 예
-------|-------|-------
map() | 각 데이터 요소에 함수를 적용해서 list를 반환 | **map(fn,data)**
filter() | 각 데이터 요소에 함수의 결과 True를 선택해서 반환 | **filter(fn, data)**
reduce() | 각 데이터 요소에 함수를 적용해서 list를 반환 | **reduce(fn, data)**


In [35]:
celsius = [39.2, 36.5, 37.3, 37.8]
def c2f(c):
    return (float(9)/5)*c + 32

f=map(c2f, celsius)
print (list(f))

[102.56, 97.7, 99.14, 100.03999999999999]


* lambda 함수의 설명
lambda는 익명함수이다. 함수의 인자를 받아서, 한 줄의 표현식으로 처리한다.

```python
lambda 함수인자 : 표현식
```

**lambda함수**를 사용하면 줄 수가 많이 줄게 된다. lambda는 무명 함수이므로 함수 선언이 별도 필요 없다. 처리 결과는 **'return'을 사용하지 않아도 반환**된다.

* lambda는 함수를 정의하는 명령어로서, 이름이 없다는 특징이 있다. 매우 직관적으로 사용할 수 있게 된다.
* map-reduce 함수에서 자주 사용되므로 잘 이해해야 한다.
* 단순히 인자에 2를 곱하는 lamdba함수 'y'를 정의하면 다음과 같다. 같은 기능을 lambda를 사용하지 않고, 기존 함수 방식으로 만들면 프로그램이 조금 더 늘어나게 된다

In [36]:
y = lambda x : x * 2
print (y(1))

2


In [37]:
def f(x):
    return x * 2
y=f(1)
print (y)

2


또는 인자를 2개 가질 수도 있다.
x, y인자를 받아 그 합산을 반환하는 lambda 함수이다.

In [38]:
x = lambda x, y : x + y
print(x(1, 2))

3


처음 map함수 결과가 object로 들어가서 list로 바꿔줘야한다.

In [39]:
f=map(lambda c:(float(9)/5)*c + 32, celsius)
print(list(f))

[102.56, 97.7, 99.14, 100.03999999999999]


In [40]:
sentence = "Hello World"
list(map(lambda x:x.split(), sentence))

[['H'], ['e'], ['l'], ['l'], ['o'], [], ['W'], ['o'], ['r'], ['l'], ['d']]

In [59]:
sentence = ["Hello World", "Good Morning"]
a = list(map(lambda x:x.split(), sentence))

print(list(map(lambda x:x.split(), sentence)))

[['Hello', 'World'], ['Good', 'Morning']]


#### filter()

filter()는 데이터를 선별한다.

x를 2로 나눈것의 나머지가 0 혹은 1 이된다. 
따라서 filter함수를 거쳐서 나머지가 1인 홀수만 적어준다. 

In [60]:
fib = [0,1,1,2,3,5,8,13,21,34,55]
result = filter(lambda x: x % 2, fib)
print (list(result))

[1, 1, 3, 5, 13, 21, 55]


#### reduce()

reduce() 역시 **함수와 데이터 2개의 인자**를 받는다.
데이터에 대해 함수를 반복적으로 적용하여 결과 값을 만들게 된다, 즉 [ func(func(s1, s2),s3), ... , sn ]와 같이 수행한다. 아래 예는 1부터 101까지 **두 수 x,y 인자를 반복해서 더한다**는 것이다.
x는 부분합계로 y를 계속 저장해 나가는 역할을 하며, 최종 합계에 이르게 된다.

단계 | x<p>부분 합계 | y | 함수적용
-----|-----|-----|-----
1 | 0 | 1 | func(0,1)
2 | 1 | 2 | func(1,2) <-- func(0 + 1, 2)
3 | 3 | 3 | func(3,3) <-- func(0 + 1 + 2, 2)
4 | 6 | 4 | func(6,4) <-- func(0 + 1 + 2 + 3, 2)
...|...|...|...


range(1,101) 뒤에 101은 포함 안된다.
x는 부분합계이고 y는 1부터 100까지 정수이다.

In [70]:
from functools import reduce
reduce(lambda x, y: x + y, range(1,101))


5050

## S.6.2 RDD 사용하기

transformation, action 함수를 사용해 보자.

### map

* 요소별 제곱

map()을 사용해서 각 요소를 **제곱**해 보자.
리스트의 요소를 하나씩 가져와서 제곱을 하게 된다.
map()은 transformation 함수라서, 실제 값은 action 함수가 적용될 때까지 연기되어 계산된다.
map() 함수의 반환 값은 RDD이다.

In [71]:
nRdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nRdd.map(lambda x: x * x)
print (squared)

PythonRDD[16] at RDD at PythonRDD.scala:53


변환의 실제 결과를 보려면 collect()를 사용해서 출력해야 한다.

In [72]:
print (squared.collect())

[1, 4, 9, 16]


* csv를 배열로

위에서 csv 파일을 읽어서 아래와 같이 구성된 요소를 정수 리스트로 만들어보자.

myRdd4는 위에 생성한 csv파일이다
리스트이다. 각각의 요소가 ''로 묶여졌다. (5개로 묶여져있다.)
형변환을 나중에 시켜줘야한다. 

In [73]:
myRdd4.take(5)

['35, 2', '40, 27', '12, 38', '15, 31', '21, 1']

2차원 리스트로 만드려면, map()함수를 사용한다.
map() 함수에는 아래와 같은 로직이 숨어있다.

```python
모든 줄을 반복:
    한 줄line을 읽는다.
    줄line을 컴마(,)로 분리한다.
    줄line을 리스트로 만든다.
```

map() 함수에 lambda를 넣어서 해보자.
line을 받아서 split(,) 즉 컴마로 분리하고 있다.

2차원 리스트로 다시 만들어졌다


In [74]:
myRdd5 = myRdd4.map(lambda line: line.split(','))
myRdd5.take(5)

[['35', ' 2'], ['40', ' 27'], ['12', ' 38'], ['15', ' 31'], ['21', ' 1']]

* 문자열을 정수로 변환

결과에서 보듯이, 리스트 안의 숫자들이 따옴표로 되어 있다.
이는 숫자가 아니라 문자라는 의미이다. 문자는 + 연산을 하면 아래와 같이 합성이 된다.

In [76]:
'35'+'2'

'352'

뭔가 연산을 하려면, 숫자를 정수로 형변환을 해야 한다.
우선 리스트의 첫 째 요소 ['35', '2']를 Python으로 형변환 해보자.
이 경우 반복문으로 하나씩 읽어서 문자를 int()로 형변환을 해야 한다.


In [78]:
x=['35', '2']
y=list()
for i in x:
    y.append(int(i))
print(y)

[35, 2]


이를 줄여서 아래와 같이 처리하는데, 프로그램이 줄 수가 줄어들어 간략해 지면서 Python의 특징이 잘 드러나고 있다.
이를 
### Pythonoic하다라고 한다.

In [79]:
x=['35', '2']
[int(i) for i in x]

[35, 2]

잘 되는 것을 확인했으니, map() 함수에 넣어서 정수로 만들자.

* 단어 분리

map 함수를 사용해서 문ㅁ서를 문장으로 분리해 보자.
문서파일이 10개 문장을 포함하고 있으므로, count()는 10개를 출력한다.

In [80]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

```str.split()``` 함수에 인자가 없으면 whitespace로 분리한다. whitespace는 공백이나 탭 등의 기호를 말하는 것으로, 문장을 분리해서 단어로 분리할 경우에 사용한다.

> whitespace

> 스페이스바, 탭, RETURN, ENTER 키를 말한다.

In [82]:
sentences=myRdd2.map(lambda x:x.split())

문서는 문장으로 문장은 단어로 구성되어있다.

In [87]:
sentences.count()


10

* 사용자 함수를 이용해서 단어 분리

사용자 함수, 즉 사용자가 만든 mySplit() 함수를 사용해 map()을 수행하고 있다.
이와 같이 lambda를 사용하지 않고, 사용자함수로 map()을 사용할 수 있다.
lambda는 한 줄의 명령문만 가지게 되므로, 여러 명령문으로 함수를 만드는 경우 사용자함수를 만들어 유용하게 사용할 수 있다.

In [88]:
def mySplit(x):
    return x.split()
sentences2=myRdd2.map(mySplit)
sentences2.count()

10

In [89]:
sentences.take(3)

[['Wikipedia'],
 ['Apache',
  'Spark',
  'is',
  'an',
  'open',
  'source',
  'cluster',
  'computing',
  'framework.'],
 ['아파치', '스파크는', '오픈', '소스', '클러스터', '컴퓨팅', '프레임워크이다.']]

* 중첩 for 문으로 2차원 배열의 출력

앞서 본 경우와 같이 리스트에는 **문장**이 분리된 **단어**가 요소로 담겨져 있으므로, **for문을 중첩**해서 사용해야 단어를 출력할 수 있다.
Python3에서는 print()가 함수로 취급된다. 단어를 출력할 경우, 그 사이에 공백을 넣고 싶으면 end= ' ' 라고 적어주어야 한다.
따옴표는 한 개 '' 또는 "" 어느 것으로 해도 기능의 차이는 없다.

첫번째 for문 문서가 문장으로 쪼개짐

두번째 for문은 문장이 단어로 쪼개짐 end때문에 공백으로 연결댐

In [90]:
for line in sentences.collect():
    for word in line:
        print (word, end=" ")
    print ("\n-----")

Wikipedia 
-----
Apache Spark is an open source cluster computing framework. 
-----
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. 
-----
Apache Spark Apache Spark Apache Spark Apache Spark 
-----
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 
-----
Originally developed at the University of California, Berkeley's AMPLab, 
-----
the Spark codebase was later donated to the Apache Software Foundation, 
-----
which has maintained it since. 
-----
Spark provides an interface for programming entire clusters with 
-----
implicit data parallelism and fault-tolerance. 
-----


* 문자 개수

각 문장의 철자 갯수를 세어 보자.
철자는 len()함수를 사용하면 된다.
첫 문장 'Wikipedia'는 **9**, 다음 문장 'Apache Spark is an open source cluster computing framework.'는 마침표를 포함하여 **59**자를 출력하고 있다.
아래는 58을 출력하는데, 자세히 살펴보면 마침표가 빠져있어서 그렇다.

In [93]:
len("Apache Spark is an open source cluster computing framework.")

59

문자 개수를 센 후, 배열로 만들어보자.


In [92]:
myRdd2.map(lambda s:len(s)).collect()

[9, 59, 32, 51, 31, 72, 71, 30, 64, 46]

* 교체

이번에는 리스트에서 RDD를 만들어 간단한 문자처리 기능으로 **대소문자 변환**이나 **교체**를 해보자.

리스트에서 rdd만들기 sparkcontext.parralelize

파일에서 rdd 만들기 sparkcontext.textfile("data","ds_spark_wiki.txt"))

In [94]:
myList=["this is","a line"]
_rdd=spark.sparkContext.parallelize(myList)

In [95]:
wordsRdd=_rdd.map(lambda x:x.split())
print (wordsRdd.collect())

[['this', 'is'], ['a', 'line']]


In [99]:
repRdd=_rdd.map(lambda x:x.replace("a","AA"))
repRdd.take(2)

['this is', 'AA line']

* 대소문자 변환

첫 글자를 대문자로 만들어서 출력해 보자.
다음 's'.upper()는 철자 's'를 대문자로 출력하는 함수이다.

In [100]:
's'.upper()

'S'

첫째 요소 **x[0]**는 리스트 각 요소의 첫째 단어 'this', 'a'를 대문자로 변환하고 있다.

In [101]:
upperRDD =wordsRdd.map(lambda x: x[0].upper())
print (upperRDD.collect())

['THIS', 'A']


리스트의 모든 단어를 대문자로 바꾸려면 for문을 사용한다.

In [102]:
upper2RDD =wordsRdd.map(lambda x: [i.upper() for i in x])
print (upper2RDD.collect())

[['THIS', 'IS'], ['A', 'LINE']]


### reduce

앞서 RDD를 사용하지 않고, Python reduce()한 예와 비교해 보자.
reduce()는 lamdba함수를 사용해서 **입력 데이터를 하나씩 서로 더해서 x+y** 결과 값을 만들어 낸다.

```python
subtotal = 0으로 초기화,
subtotal = subtotal + 1 (y는 1이므로)
subtotal = subtotal + 2 (y는 하나 증가해서 2이므로)
...
subtotal = subtotal + 100 (y는 하나씩 증가해서 최종 값 100)
```

1부터 100까지 합 구하기

1. 반복분

2. 재귀함수 

3. reduce

In [103]:
myRdd100 = spark.sparkContext.parallelize(range(1,101))
myRdd100.reduce(lambda subtotal, x: subtotal + x)

5050

### 단순 통계 기능

텍스트데이터와 달리 정량데이터로부터 sum, min, max, 표준편차 등 서술통계를 계산해 낼 수 있다.

In [104]:
print ("sum: ", myRdd100.sum())
print ("min: ", myRdd100.min())
print ("max: ", myRdd100.max())
print ("standard deviation:", myRdd100.stdev())
print ("variance: ", myRdd100.variance())

sum:  5050
min:  1
max:  100
standard deviation: 28.86607004772212
variance:  833.25


### filter()

filter() 함수로 조건에 맞는 문장만 분리해 보자.
"Spark" 단어가 포함된 문장이 조건이된다.
count() 함수로 그 갯수를 확인해 보자.

In [105]:
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line)
print ("How many lines having 'Spark': ",myRdd_spark.count())

How many lines having 'Spark':  4


유니코드 일때는 u자를 넣어줘야한다. 

In [106]:
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print (myRdd_unicode.first())

아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.


* filter()를 사용해서 **stopwords** 제거하기

문장 안에 stopwords를 포함한 경우는 제거되지 않는다.
따라서 flatMap()을 하고 단어에 대해 불용어를 제거해야 한다.
불용어는 단어빈도를 계산하면서 제거하고 싶은 단어를 말한다.
불용어는 빈도를 세어도 의미가 없는 대명사 (이, 그, 저...) 또는 한 글자 단어 (등...)이 될 수 있다.
한글은 유니코드로 처리해야 한다.
영어는 대소문자를 모두 처리하기 위해 여기서는 소문자로 만들어 처리한다.

flat 2차원 구조를 1차원으로 만들어야한다.

In [107]:
stopwords = ['is','am','are','the','for','a', 'an', 'at']
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

In [108]:
for words in myRdd_stop.collect():
    print (words, end=' ')

Wikipedia Apache Spark open source cluster computing framework. 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. Apache Spark Apache Spark Apache Spark Apache Spark 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 Originally developed University of California, Berkeley's AMPLab, Spark codebase was later donated to Apache Software Foundation, which has maintained it since. Spark provides interface programming entire clusters with implicit data parallelism and fault-tolerance. 

### foreach()

```foreach()```는 action이지만, 다른 action 함수들과 달리 **반환 값이 없다**.
각 요소에 대해 적용한다는 역할에 대해 유사한 기능을 하는 ```map()``` 함수가 있다.
```map()``` 함수는 각 요소에 대해 계산을 하고, 그 값을 반환한다.

In [109]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)

In [110]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

노트북에서는 안댐 python에서만 됨

In [112]:
def f(x): print(x)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(f)

### pipeline

파이프라인은 **transformation**(예: map()), **action**(예: collect()) 함수를 **연이어 적용**하는 방식을 말한다.
파이프라인이 아니라면 함수를 하나씩 끝나고, 결과를 받은 후 다음 함수를 단계별로 적용하게 된다.
보다 효율적인 처리를 위해 함수들을 파이프라인같이 붙여서 중간결과를 별도로 산출하지 않고 연이어 처리한다. 

In [113]:
upper2list=wordsRdd.map(lambda x: [i.upper() for i in x]).collect()
print (type(upper2list))

<class 'list'>


### 파일에 쓰기

* RDD 저장
Spark에서 list는 RDD로 만들어 로컬 파일에 쓰게 된다.

In [115]:
spark.sparkContext.parallelize(upper2list).saveAsTextFile("data/ds_spark_wiki_out_1")

In [119]:
!ls data/ds_spark_wiki_out*

data/ds_spark_wiki_out:
_SUCCESS   part-00000

data/ds_spark_wiki_out_1:
_SUCCESS   part-00000


In [120]:
_rdd=spark.sparkContext.textFile("data/ds_spark_wiki_out")
_rdd.collect()

["['THIS', 'IS']", "['A', 'LINE']"]

방금 작성된 파일을 읽어서 내용을 읽어보는 것도 물론 가능하다.

파일로 쓰려면 ```coalesce()``` 함수를 사용해야 한다. 인자는 partition의 수를 적어 준다.
```python
_rdd.coalesce(1).saveAsTextFile("data/ds_spark_wiki_txt")
```

In [126]:
_rdd.map(lambda x: "".join(x)).coalesce(1).saveAsTextFile("data/ds_spark_wiki_out_txt")

In [131]:
!cat data/ds_spark_wiki_txt/part-00000

['THIS', 'IS']
['A', 'LINE']


### groupBy

transformation 함수이다. Paired, unpaired RDD에 모두 사용할 수 있지만, 주로 **unpaired RDD**에 많이 쓰인다.
키를 선택하여 사용할 수 있는 점이 장점이다.

Paired 데이터는 예를 들면 아래와 같이 key, value 쌍으로 구성된 데이터를 말한다.

```python
(key=202011111, ('lim', 'computer'))
(key=202011112, ('cho', 'computer'))
(key=202011113, ('lim', 'business'))
```
반면에 Unpaired는 key, value 쌍으로 구성이 되지 않은 데이터를 말한다. 아래는 이름, 전공으로 구성된 데이터로 키가 없다.
또한 텍스트로 구성된 데이터도 키가 없으므로 unpaired 데이터이다.

```python
'lim', 'computer'
'cho', 'computer'
'lim', 'business'
```

유사한 기능을 수행하는 **groupByKey()와 비교해서 상대적으로 빠르지 않다.**

In [132]:
myRdd2.take(10)

['Wikipedia',
 'Apache Spark is an open source cluster computing framework.',
 '아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 '아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크',
 "Originally developed at the University of California, Berkeley's AMPLab,",
 'the Spark codebase was later donated to the Apache Software Foundation,',
 'which has maintained it since.',
 'Spark provides an interface for programming entire clusters with',
 'implicit data parallelism and fault-tolerance.']

* 앞 2글자로 groupBy

앞 2글자를 key로 사용해서, groupBy()를 사용할 수 있다.
결과는 key-value 쌍으로 만들어졌는데, key는 예상한대로 되었으나 value는 iterator로 생성되었다.

두글자만 띄어서 그룹바이 해줌

In [133]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    print ("{}: {}".format(k, v))

Wi: <pyspark.resultiterable.ResultIterable object at 0x7f939a06a6a0>
Ap: <pyspark.resultiterable.ResultIterable object at 0x7f939a06a2e0>
아파: <pyspark.resultiterable.ResultIterable object at 0x7f939a06ab80>
Or: <pyspark.resultiterable.ResultIterable object at 0x7f9399e72190>
th: <pyspark.resultiterable.ResultIterable object at 0x7f9399e72ca0>
wh: <pyspark.resultiterable.ResultIterable object at 0x7f9399e729d0>
Sp: <pyspark.resultiterable.ResultIterable object at 0x7f9398e90a60>
im: <pyspark.resultiterable.ResultIterable object at 0x7f9398e90460>


ResultIterable은 반복문으로 해체하여 결과를 출력하면 된다.

In [134]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    for eachValue in v:
        print ("{}: {}".format(k, eachValue))
    print ("-----")

Wi: Wikipedia
-----
Ap: Apache Spark is an open source cluster computing framework.
Ap: Apache Spark Apache Spark Apache Spark Apache Spark
-----
아파: 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
아파: 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
-----
Or: Originally developed at the University of California, Berkeley's AMPLab,
-----
th: the Spark codebase was later donated to the Apache Software Foundation,
-----
wh: which has maintained it since.
-----
Sp: Spark provides an interface for programming entire clusters with
-----
im: implicit data parallelism and fault-tolerance.
-----


* (key, value)로 구성된 Paired 데이터의 groupBy

다음 데이터에 groupBy()를 적용해 보자.

In [135]:
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]

우선 SparkContext parallelize() 함수를 사용하여 리스트에서 RDD를 생성한다.

In [136]:
_testRdd=spark.sparkContext.parallelize(_testList)

* ResultIterable

첫 요소 x[0]는 key이고, 두 개의 key1, key2가 있다. key를 groupBy()로 구분하여 묶어 보자.
groupByKey()의 결과는 **```ResultIterable```**이고, 이 객체는 그대로 볼 수 없다.

In [137]:
_testRdd.groupBy(lambda x:x[0]).collect()

[('Seoul', <pyspark.resultiterable.ResultIterable at 0x7f939a061400>),
 ('Busan', <pyspark.resultiterable.ResultIterable at 0x7f939a0b9df0>)]

* mapValues

ResultIterable을 리스트로 변환하여 값을 보기로 한다.
이 경우 **```mapValues()```** 함수를 사용한다.
이 함수를 사용하여, 각 키 (key1, key2)에 대해 **values 값**들의 리스트를 수집하고 있다.

mapValues-> key에 대한 value 들어감

또는 보다 간단하게 다음과 같이 ```list```만을 해주어도 된다.

In [138]:
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

### 5.6.3 Pair RDD

Pair RDD는 **key,value 쌍**으로 구성된 RDD를 말한다.
이러한 RDD는 키에 대해 연산을 하는 **byKey()** 또는 값에 대해 **byValue()** 함수를 사용할 수 있다.

구분 | 설명
-----|-----
byKey 연산 | 동일한 키에 대해 연산<br>- 단계 1: key-value를 계산한다. 각 key의 빈도를 계산하여  '(key,1)' 형식으로 만든다.<br>- 단계 2: byKey를 적용한다. 동일한 key의 value를 더해준다.
byValue 연산 | 그룹으로 구분하고 나면, 값이 복수 개가 된다. 복수의 값에 대해 개수를 셀지, 합계를 계산할지 mapValues() 함수에서 하게 된다.


다음과 같이 groupByKey(), reduceBykey(), combineByKey(), aggregateByKey(), mapValues() 함수를 사용할 수 있다.

구분 | 설명
-----|-----
groupByKey() | 같은 key를 grouping, partition에서 **먼저 reduce하지 않고, 전체로 계산**한다.
reduceByKey() | 같은 key의 value를 합계, partition에서 **먼저 reduce하고, 전체로 계산**한다. grouping + aggregation.<p>즉 **reduceByKey = groupByKey().reduce()**
combineByKey() | 키별로 합계, 개수 (key, (sum, count))를 계산. createCombiner, mergeValue, mergeCombiners.
aggregateByKey() | reduceByKey()와 유사한 기능을 수행한다. 키별로 합계, 개수, 평균을 계산.
mapValues() | Paired RDD는 key,value가 있기 마련이고, **value에 적용하는 함수**이다.

#### Paired RDD 데이터 예제

Partition1, 2, 3으로 데이터가 분할되어 있다고 가정하자.

P1 | P2 | P3
-----|-----|-----
(key1,1)<br> | (key1,1)<br> | (key1,1)<br>
(key1,1)<br> | (key2,1)<br> | (key1,1)<br>
(key1,1)<br> |              | (key2,1)<br>
(key2,1)<br> |              | (key2,1)<br>
(key2,1)<br> |              |

#### Paired RDD 생성

테스트 데이터를 구성하고, parallelize()함수를 사용하여 RDD를 만든다.